In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
from selenium.webdriver.common.keys import Keys

In [ ]:
if 'data' not in os.listdir():
    os.mkdir('data') 

## 현재 진행중인 이벤트 크롤링

In [ ]:
browser = webdriver.Chrome()

In [155]:
url = 'https://maplestory.nexon.com/News/Event/Ongoing'
browser.get(url)

In [156]:
result=[]
i=0
while True:
    event_button_list = browser.find_elements('css selector', 'div.event_board > ul >li')
    event_button_list[i].click()

    j=0
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.select('p.qs_title>span')[0].text
    duration = soup.select('div.qs_info_wrap > span.event_date')[0].text
    click_num = soup.select('div.qs_info>p')[0].text
    reply_num = soup.select('div.reply_title>h2>span')[0].text
    
    while True:
        reply_button_list = browser.find_elements('css selector','div.page_numb2>a')
        reply_button_list[j].click()
        reply = soup.select('div.reply>p.reply_text')
        for r in reply:
            data= [title, duration, click_num, reply_num, r.text]
            result.append(data)
        time.sleep(1)
        j+=1
        if j==len(reply_button_list):
            break
    browser.get(url)
    i+=1
    if i==len(event_button_list):
        break
    time.sleep(1)

In [161]:
df = pd.DataFrame(result)
df.columns = ['title', 'duration', 'click_num', 'reply_num', 'reply']
df.to_csv("data/ongoing_event.csv", index=False)

## 썬데이 메이플 크롤링

In [2]:
browser = webdriver.Chrome()

In [3]:
url = 'https://www.fmkorea.com/index.php?mid=maple&category=1135601723'
browser.get(url)

In [20]:
from selenium.webdriver.support.ui import Select
input_text = browser.find_elements('css selector', 'input#bd_srch_btm_itx_1135415169')[0]
drop_down = browser.find_element('css selector','span.btn_img select')
search = "썬데이 메이플(2023)"

input_text.send_keys(search)
select = Select(drop_down)
select.select_by_value('title')
input_text.send_keys(Keys.RETURN)

In [21]:
while True:
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    now = soup.select('strong.direction')[0].text.strip()
    title_reply_list = soup.select('td.title.hotdeal_var8 > a')
    m_no_list = soup.select('td.m_no')
    for idx, t_r in enumerate(title_reply_list):
        if idx % 2==0:
            title = t_r.text.strip()
            m_no = m_no_list[idx].text.strip()
        else:
            reply_num = t_r.text
            m_no_voted = m_no_list[idx].text.strip()
    if now=='다음':
        break
    else:
        button = browser.find_elements('css selector','a.direction')[0]
        button.click()
print(title, reply_num, m_no, m_no_voted)

썬데이 메이플 (2023년 1월 1일) 20 5798 19
